In [ ]:
# 이 블록은 빼서 돌릴 것
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('cuda')
else:
    device = torch.device('cpu')
    print('cpu')

cuda


In [2]:
import numpy as np
f = open('live_record(1.567G, 50MHz, 25dB, 5M)_230513_2013_102s_SDR_검증.bin', 'rb')
data = np.fromfile(f, dtype=np.int16, count = int(50e6*10*2) + 4000)
f.close()

# 오류로 인한 값들 삭제
sig = []
signal_data = []

# 시간 복잡도를 줄인 배열 요소 삭제 알고리즘
for i in range(2,len(data),500002):
# i = 2, 500004, 1000006, ... (첫 두 개의 값도 spike) 
    frac = data[i:i+500000] # 50만 개씩 slicing
    real = frac[::2]
    imag = frac[1::2]
    signal_data.append(real + 1j * imag) # 자른 값을 결과에 추가
    
# 결과 sig 배열을 1차원으로 변경
signal_data = np.hstack(signal_data)

In [ ]:
# C/A Code Generation
from CA_code_generator import *

code = CA_code(50e6)
code.set_all_settings()

codes = code.get_All_Codes()
ov_codes = code.get_All_Oversampled_Codes()
sample_count = code.sample_per_chip

In [ ]:
# Acquisition
from Acquisition import *
acq = acquisition(8.42e6, codes, sample_count)
svResult = acq.perform(signal_data)

Checking...
----------------------------------------
SV 5 Found.
Estimated code delay : 164.0
Estimated frequency offset : 500.0
----------------------------------------
SV 11 Found.
Estimated code delay : 628.0
Estimated frequency offset : -2000.0
----------------------------------------
SV 13 Found.
Estimated code delay : 269.0
Estimated frequency offset : 1500.0
----------------------------------------
SV 15 Found.
Estimated code delay : 83.5
Estimated frequency offset : 3000.0
----------------------------------------
SV 20 Found.
Estimated code delay : 116.0
Estimated frequency offset : -500.0
----------------------------------------
SV 29 Found.
Estimated code delay : 970.5
Estimated frequency offset : -500.0
----------------------------------------
SV 30 Found.
Estimated code delay : 844.0
Estimated frequency offset : 0.0
----------------------------------------
Found SV :5 11 13 15 20 29 30 


In [ ]:
# Tracking
from Tracking import *

t = tracking(signal_data, 8.42e6, svResult[:5], ov_codes)
t.process()
svBits = t.getBit()

Processing... 1/5
Processing... 2/5
Processing... 3/5
Processing... 4/5
Processing... 5/5
Done


In [ ]:
# Generate text file from bit sequence 
for i, row in enumerate(svBits):
    filename = f"bit_sequence_row_{i+1}.txt"  # 각 파일명에 행 번호 추가
    with open(filename, 'w') as f:
        row_str = ''.join(map(str, row))  # 이진 배열을 문자열로 변환
        f.write(row_str)  # 파일에 쓰기